In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import timeit
import re             # regular expression library
from wordcloud import WordCloud
from wordcloud import WordCloud,STOPWORDS
from nltk.corpus import stopwords
from gensim.corpora.dictionary import Dictionary
from nltk.tokenize import word_tokenize
from collections import Counter
from gensim.models.phrases import Phrases, Phraser
#from gensim.corpora.dictionary import Dictionary
from gensim.models import TfidfModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [100]:
pwd

'/Users/Robert/DSI/Capstone_3'

In [101]:
# Read datasets/papers.csv into papers
papers = pd.read_csv('nips-papers/papers.csv')


In [102]:
papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [103]:
papers.tail()

,id,year,title,event_type,pdf_name,abstract,paper_text
7236,994,1994,Single Transistor Learning Synapses,NaN,994-single-transistor-learning-synapses.pdf,Abstract Missing,Single Transistor Learning Synapses\n\nPaul Ha...
7237,996,1994,"Bias, Variance and the Combination of Least Sq...",NaN,996-bias-variance-and-the-combination-of-least...,Abstract Missing,"Bias, Variance and the Combination of\nLeast S..."
7238,997,1994,A Real Time Clustering CMOS Neural Engine,NaN,997-a-real-time-clustering-cmos-neural-engine.pdf,Abstract Missing,A Real Time Clustering CMOS\nNeural Engine\nT....
7239,998,1994,Learning direction in global motion: two class...,NaN,998-learning-direction-in-global-motion-two-cl...,Abstract Missing,Learning direction in global motion: two\nclas...
7240,999,1994,Correlation and Interpolation Networks for Rea...,NaN,999-correlation-and-interpolation-networks-for...,Abstract Missing,Correlation and Interpolation Networks for\nRe...


In [104]:
papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7241 entries, 0 to 7240
Data columns (total 7 columns):
id            7241 non-null int64
year          7241 non-null int64
title         7241 non-null object
event_type    2422 non-null object
pdf_name      7241 non-null object
abstract      7241 non-null object
paper_text    7241 non-null object
dtypes: int64(2), object(5)
memory usage: 396.1+ KB


In [105]:
drop_cols = ['id', 'event_type', 'pdf_name']
papers.drop(columns=drop_cols, axis=1, inplace=True)


In [106]:
# Make dataframe for each half decade 1987 - 2017
papers_thru_1990 = papers[papers.year <= 1990]
papers_thru_1990.reset_index(inplace=True)

papers_thru_1995 = papers[(papers.year > 1990) & (papers.year <= 1995)]
papers_thru_1995.reset_index(inplace=True)

papers_thru_2000 = papers[(papers.year > 1995) & (papers.year <= 2000)]
papers_thru_2000.reset_index(inplace=True)

papers_thru_2005 = papers[(papers.year > 2000) & (papers.year <= 2005)]
papers_thru_2005.reset_index(inplace=True)

papers_thru_2010 = papers[(papers.year > 2005) & (papers.year <= 2010)]
papers_thru_2010.reset_index(inplace=True)

papers_thru_2015 = papers[(papers.year > 2010) & (papers.year <= 2015)]
papers_thru_2015.reset_index(inplace=True)

papers_thru_2017 = papers[(papers.year > 2015) & (papers.year <= 2017)]
papers_thru_2017.reset_index(inplace=True)


# Preprocess text

In [107]:
# Use stopwords from "nltk.corpus"
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) # sets are efficient
stop_words.update(["this", "that", "thus", "from", "does", "example", "however", "since", "given", 
                    "et", "al", "hence", "therefore", "use", "used", "note", "corresponding", 
                    "setting", "well", "one", "form", "using", "second", "even", "known", "either",
                    "consider", "particular", "general", "represent", "case", "output", "defined",
                    "rather", "though", "although", "set", "problem", "function", "figure", "given",
                    "results", "number", "time", "data", "results", "given", "information", "different",
                    "models", "method", "n", "j", "r", "l", "mj", "a", "cj", "zero", "iv", "is", "elsewhere",
                    "large", "may", "finally", "c", "describing", "l", "cells", "single", "field",
                    "approximation", "activity", "mean", "input"])


In [108]:
start_time = timeit.default_timer()
# This function removes process the stopwords. Takes 3-4 mins
def rem_stopwords(txt):
    words = txt.split()
    ret_word = words.copy()
    for w in words: 
        if w.lower() in stop_words:
            ret_word.remove(w)
    return (" ".join(ret_word))
papers_thru_2017['text_processed'] = papers_thru_2017['paper_text'].map(rem_stopwords)
elapsed = timeit.default_timer() - start_time
print(elapsed)

71.30657320699993


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [109]:
start_time = timeit.default_timer()
# This cell takes a long time, longer than 3 mins(seems slower)
#Print the text of the first 3 rows 
print(papers_thru_2017['paper_text'].head(3))

# Remove punctuation
papers_thru_2017['text_processed'] = papers_thru_2017['text_processed'].map(lambda text: [w for w in word_tokenize(text.lower()) 
                  if w.isalpha()])

# # Print the processed titles of the first rows 
papers_thru_2017['text_processed'][0:3]

elapsed = timeit.default_timer() - start_time
print(elapsed)

0    Optimistic Gittins Indices\nEli Gutin\nOperati...
1    Sub-sampled Newton Methods\nwith Non-uniform S...
2    Budgeted stream-based active learning\nvia ada...
Name: paper_text, dtype: object
51.30639551400054


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [110]:
#Runs fast, less than 2 secs.
# start_time = timeit.default_timer()
papers_thru_2017['text_string'] = [" ".join(map(str, l)) for l in papers_thru_2017['text_processed']]
elapsed = timeit.default_timer() - start_time
# print(elapsed)
# papers.head(2)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [111]:
print(papers_thru_2017.info())
papers_thru_2017.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1248 entries, 0 to 1247
Data columns (total 7 columns):
index             1248 non-null int64
year              1248 non-null int64
title             1248 non-null object
abstract          1248 non-null object
paper_text        1248 non-null object
text_processed    1248 non-null object
text_string       1248 non-null object
dtypes: int64(2), object(5)
memory usage: 68.3+ KB
None


,index,year,title,abstract,paper_text,text_processed,text_string
0,5565,2016,Optimistic Gittins Indices,"Starting with the Thomspon sampling algorithm,...",Optimistic Gittins Indices\nEli Gutin\nOperati...,"[optimistic, gittins, indices, eli, gutin, ope...",optimistic gittins indices eli gutin operation...
1,5566,2016,Sub-sampled Newton Methods with Non-uniform Sa...,We consider the problem of finding the minimiz...,Sub-sampled Newton Methods\nwith Non-uniform S...,"[newton, methods, sampling, peng, xu, jiyan, y...",newton methods sampling peng xu jiyan yang far...
2,5567,2016,Budgeted stream-based active learning via adap...,Active learning enables us to reduce the annot...,Budgeted stream-based active learning\nvia ada...,"[budgeted, active, learning, via, adaptive, su...",budgeted active learning via adaptive submodul...


In [112]:
print(papers.info())
papers.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7241 entries, 0 to 7240
Data columns (total 4 columns):
year          7241 non-null int64
title         7241 non-null object
abstract      7241 non-null object
paper_text    7241 non-null object
dtypes: int64(1), object(3)
memory usage: 226.4+ KB
None


,year,title,abstract,paper_text
0,1987,Self-Organization of Associative Database and ...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,1987,A Mean Field Theory of Layer IV of Visual Cort...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,1988,Storing Covariance by the Associative Long-Ter...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...


In [113]:
#print(list(papers_thru_2017.text_processed[0]))
type(papers_thru_2017.text_processed)

pandas.core.series.Series

In [114]:
# # Use stopwords from "nltk.corpus"
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english')) # sets are efficient
STOPWORDS.update(["this", "that", "thus", "from", "does", "example", "however", "since", "given", 
                    "et", "al", "hence", "therefore", "use", "used", "note", "corresponding", 
                    "setting", "well", "one", "form", "using", "second", "even", "known", "either",
                    "consider", "particular", "general", "represent", "case", "output", "defined",
                    "rather", "though", "although", "set", "problem", "function", "figure", "given",
                    "results", "number", "time", "data", "results", "given", "information", "different",
                    "models", "method", "n", "j", "r", "l", "mj", "a", "cj", "zero", "iv", "is", "elsewhere",
                    "large", "may", "finally", "c", "describing", "l", "cells", "single", "field",
                    "approximation", "activity", "mean", "input", "x", "f", "here", "y", "means", "make", "say",
                    "that", "then", "good", "clarify", "b", "t", "layers", "s", "e", "units", "unit", "input", "h", 
                    "z", "p", "xt", "items", "player", "two", "item", "let", "d", "w", "ti", "m", "g", "local",
                    "q", "actions", "users", "system", "regret", "user", "xi", "rule",
                    "eq", "kl", "em", "v", "u", "first", "entries", "optimal", "error", "o", "result",
                    "see", "section", "update", "theorem", "variables", "rl", "st", "size", "based", "fig", "source",
                    "sources", "step", "o", "xi", "vi", "approach", "yt", "c", "new", "unit", "systems", "points", "point",
                    "test", "values", "value", "fi", "examples", "representation", "processing", "shown", "inputs", "initial",
                    "many", "problems", "speaker", "yi", "effect", "gp"])


In [115]:
print(len(STOPWORDS))
STOPWORDS

337


{'a',
 'about',
 'above',
 'actions',
 'activity',
 'after',
 'again',
 'against',
 'al',
 'all',
 'also',
 'although',
 'am',
 'an',
 'and',
 'any',
 'approach',
 'approximation',
 'are',
 "aren't",
 'as',
 'at',
 'b',
 'based',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'c',
 'can',
 "can't",
 'cannot',
 'case',
 'cells',
 'cj',
 'clarify',
 'com',
 'consider',
 'corresponding',
 'could',
 "couldn't",
 'd',
 'data',
 'defined',
 'describing',
 'did',
 "didn't",
 'different',
 'do',
 'does',
 "doesn't",
 'doing',
 "don't",
 'down',
 'during',
 'e',
 'each',
 'effect',
 'either',
 'else',
 'elsewhere',
 'em',
 'entries',
 'eq',
 'error',
 'et',
 'even',
 'ever',
 'example',
 'examples',
 'f',
 'few',
 'fi',
 'field',
 'fig',
 'figure',
 'finally',
 'first',
 'for',
 'form',
 'from',
 'function',
 'further',
 'g',
 'general',
 'get',
 'given',
 'good',
 'gp',
 'h',
 'had',
 "hadn't",
 'has',
 "hasn't",
 'have',
 "haven't",
 'having',


In [116]:
def tokenize(text):
    return [token for token in text if token not in STOPWORDS]
paps = papers_thru_2017.text_processed
documents = paps.tolist()
texts = [tokenize(document) for document in documents]

In [117]:
print(type(texts))
len(texts)

<class 'list'>


1248

In [118]:
trg_sent = [['machine','learning','neural','network','dummy','neural','networks', "deep", "learning"], 
            ['neural','networks', 'dummy','neural','network','machine','learning',"deep", "learning"]]
phrases = Phrases(trg_sent, min_count=1, threshold=1)
bigram = Phraser(phrases)
test_sent = ['machine','learning','neural','network','neural','networks', "deep", "learning"]
print(bigram[test_sent])

corpus = [bigram[paper] for paper in texts]

['machine_learning', 'neural_network', 'neural_networks', 'deep_learning']


In [119]:
print(texts[5])
corpus[111]

['tagging', 'markov', 'chain', 'optimization', 'nir', 'rosenfeld', 'school', 'computer', 'science', 'engineering', 'hebrew', 'university', 'jerusalem', 'amir', 'globerson', 'blavatnik', 'school', 'computer', 'science', 'tel', 'aviv', 'university', 'gamir', 'abstract', 'tags', 'keywords', 'describe', 'annotate', 'content', 'allow', 'efficient', 'organization', 'categorization', 'facilitate', 'relevant', 'search', 'queries', 'selected', 'tags', 'considerable', 'volume', 'traffic', 'eventually', 'reaches', 'tagging', 'tags', 'exclusively', 'chosen', 'owner', 'turn', 'interested', 'maximizing', 'traffic', 'principled', 'assigning', 'tags', 'prove', 'valuable', 'paper', 'introduce', 'tagging', 'task', 'choose', 'subset', 'tags', 'probability', 'browsing', 'reaching', 'maximized', 'formulate', 'modeling', 'traffic', 'markov', 'chain', 'asking', 'transitions', 'chain', 'modified', 'maximize', 'traffic', 'certain', 'state', 'interest', 'resulting', 'optimization', 'involves', 'maximizing', 'ce

['towards',
 'unifying',
 'hamiltonian',
 'monte',
 'carlo',
 'slice',
 'sampling',
 'yizhe',
 'zhang',
 'xiangyu',
 'wang',
 'changyou',
 'chen',
 'ricardo',
 'henao',
 'kai',
 'fan',
 'lawrence',
 'carin',
 'duke',
 'university',
 'durham',
 'nc',
 'lcarin',
 'abstract',
 'unify',
 'slice',
 'sampling',
 'hamiltonian',
 'monte',
 'carlo',
 'hmc',
 'sampling',
 'demonstrating',
 'connection',
 'via',
 'equation',
 'hamiltonian',
 'mechanics',
 'insight',
 'enables',
 'extension',
 'hmc',
 'slice',
 'sampling',
 'broader',
 'family',
 'samplers',
 'called',
 'monomial',
 'gamma',
 'samplers',
 'mgs',
 'provide',
 'theoretical',
 'analysis',
 'mixing',
 'performance',
 'samplers',
 'proving',
 'limit',
 'parameter',
 'mgs',
 'draws',
 'decorrelated',
 'samples',
 'desired',
 'target',
 'distribution',
 'show',
 'parameter',
 'tends',
 'toward',
 'limit',
 'performance',
 'gains',
 'achieved',
 'cost',
 'increasing',
 'numerical',
 'difficulty',
 'practical',
 'convergence',
 'issues',
 

In [120]:
corpus[2]

['budgeted',
 'active',
 'learning',
 'via',
 'adaptive',
 'submodular',
 'maximization',
 'kaito',
 'fujii',
 'kyoto',
 'university',
 'jst',
 'erato',
 'kawarabayashi',
 'graph',
 'project',
 'fujii',
 'hisashi',
 'kashima',
 'kyoto',
 'university',
 'kashima',
 'abstract',
 'active',
 'learning',
 'enables',
 'us',
 'reduce',
 'annotation',
 'cost',
 'adaptively',
 'selecting',
 'unlabeled',
 'instances',
 'labeled',
 'active',
 'learning',
 'several',
 'effective',
 'methods',
 'theoretical',
 'guarantees',
 'developed',
 'maximizing',
 'utility',
 'satisfying',
 'adaptive',
 'submodularity',
 'contrast',
 'methods',
 'active',
 'learning',
 'adaptive',
 'submodularity',
 'paper',
 'propose',
 'class',
 'utility',
 'functions',
 'submodular',
 'functions',
 'includes',
 'existing',
 'adaptive',
 'submodular',
 'functions',
 'appearing',
 'real',
 'world',
 'provide',
 'framework',
 'submodularity',
 'makes',
 'possible',
 'convert',
 'existing',
 'poolbased',
 'methods',
 'methods'

In [121]:
len(corpus)

1248

In [122]:
from collections import defaultdict
frequency = defaultdict(int)
for text in corpus:
    for token in text:
        frequency[token] += 1

texts_2 = [[token for token in text if frequency[token] > 10] for text in corpus]

In [123]:
papers_thru_2017.head(3)

,index,year,title,abstract,paper_text,text_processed,text_string
0,5565,2016,Optimistic Gittins Indices,"Starting with the Thomspon sampling algorithm,...",Optimistic Gittins Indices\nEli Gutin\nOperati...,"[optimistic, gittins, indices, eli, gutin, ope...",optimistic gittins indices eli gutin operation...
1,5566,2016,Sub-sampled Newton Methods with Non-uniform Sa...,We consider the problem of finding the minimiz...,Sub-sampled Newton Methods\nwith Non-uniform S...,"[newton, methods, sampling, peng, xu, jiyan, y...",newton methods sampling peng xu jiyan yang far...
2,5567,2016,Budgeted stream-based active learning via adap...,Active learning enables us to reduce the annot...,Budgeted stream-based active learning\nvia ada...,"[budgeted, active, learning, via, adaptive, su...",budgeted active learning via adaptive submodul...


In [124]:
len(texts_2)

1248

In [125]:
from gensim import corpora
dictionary = corpora.Dictionary(texts_2)
corpus_test = [dictionary.doc2bow(text) for text in texts_2]

In [126]:
len(corpus_test)

1248

In [127]:
len(dictionary)

15219

In [128]:
start_time = timeit.default_timer()
from gensim.models.ldamodel import LdaModel
# fit gensim's LDA model
# the number of topics should be optimized iteratively
papers_topics = LdaModel(corpus=corpus_test,
                           id2word=dictionary,
                           num_topics=20,
                           passes=10)


elapsed = timeit.default_timer() - start_time
print(elapsed)

89.90343281500009


In [129]:
from pprint import pprint
# print out first 10 topics
for i, topic in enumerate(papers_topics.print_topics(20)):
    pprint("{} --- {}".format(i, topic))
    print()


('0 --- (0, \'0.019*"model" + 0.007*"neural" + 0.005*"learning" + '
 '0.004*"neurons" + 0.003*"performance" + 0.003*"features" + 0.003*"brain" + '
 '0.003*"object" + 0.003*"inference" + 0.003*"network"\')')

('1 --- (1, \'0.016*"variational" + 0.012*"inference" + 0.010*"model" + '
 '0.009*"posterior" + 0.009*"log" + 0.008*"distribution" + 0.007*"gradient" + '
 '0.006*"bayesian" + 0.005*"parameters" + 0.005*"dropout"\')')

('2 --- (2, \'0.013*"kernel" + 0.008*"random" + 0.007*"clustering" + '
 '0.006*"log" + 0.006*"model" + 0.006*"queries" + 0.006*"algorithm" + '
 '0.005*"probability" + 0.005*"learning" + 0.004*"query"\')')

('3 --- (3, \'0.013*"submodular" + 0.012*"algorithm" + 0.010*"functions" + '
 '0.006*"greedy" + 0.005*"optimization" + 0.005*"loss" + 0.005*"learning" + '
 '0.005*"model" + 0.004*"hash" + 0.004*"algorithms"\')')

('4 --- (4, \'0.016*"model" + 0.010*"generative" + 0.009*"latent" + '
 '0.009*"training" + 0.009*"distribution" + 0.007*"learning" + 0.006*"samples" '
 '+ 

In [130]:
# Takes about 5-10 minutes, depends how many papers
import pyLDAvis.gensim as gensimvis
import pyLDAvis
vis_data = gensimvis.prepare(papers_topics, corpus_test, dictionary)
pyLDAvis.display(vis_data)

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
